In [6]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving fluxodecaixa_mil.csv to fluxodecaixa_mil (1).csv
Saving parametros_beta.csv to parametros_beta (1).csv


In [36]:

''' 
Definindo a classe Valuation para facilitar a simulação de diferentes cenários.
Parâmetros obrigatórios: inflação projetada e crescimento do PIB projetado.
Parâmetros opcionais: ano inicial, ano final, número de anos projetados e crescimento da receita. Caso estes não sejam definidos, serão gerados pelo próprio programa.

1º passo: importar os dados das demonstrações financeiras patrimoniais.
2º passo: calcular o WACC, com parâmetros importados via csv.
3º passo: calcular o preço justo.
''' 
class Valuation():
    def __init__(self, inflacao_projetada: float, crescimento_pib: float, ano_inicial: int=None, ano_final: int=None, anos_projetados: int=None, crescimento_receita: float=None):

        self.tributacao = 0.34

        self.anoInicial = ano_inicial  
        self.anoFinal = ano_final
        self.nAnosProjetados = anos_projetados

        self.crescimentoReceita = crescimento_receita

        self.inflacao = inflacao_projetada
        self.crescimento_pib = crescimento_pib

        print('\tINFORMAÇÕES DFP\n')
        self.importarDFP()
        print('\tCÁLCULO WACC\n')
        self.calculoWACC()
        print('\tCÁLCULO PREÇO JUSTO\n')
        self.calculoPrecoJusto()
    

    def importarDFP(self):
        
        ###IMPORTANDO CSV DAS DFPs.###        
        fluxo = pd.read_csv('fluxodecaixa_mil (1).csv', sep=';', encoding='utf-8')
        #fluxo = pd.read_csv(r'C:\Users\marios\Documents\mng\fluxodecaixa_mil.csv', sep=';', encoding='utf-8')
        fluxo.sort_values(by='Ano', ascending=True, inplace=True)
        fluxo.reset_index(drop=True, inplace=True)

        '''Definindo janela temporal do valuation'''
        if self.anoInicial is None:
            self.anoInicial = fluxo['Ano'].min()
        if self.anoFinal is None:
            self.anoFinal = fluxo['Ano'].max()
        if self.nAnosProjetados is None:
            self.nAnosProjetados = 5

        print(f'Ano inicial: {self.anoInicial}\n'
            f'Ano final: {self.anoFinal}\n'        
            )

        ###CRIAR TABELAS COM OUTROS INDICADORES EM RELAÇÃO À RECEITA(=1)###
        tabela_projecao = pd.DataFrame()

        tabela_projecao['Ano'] = fluxo['Ano']
        tabela_projecao['Receita'] = 1
        tabela_projecao['EBIT'] = fluxo['EBIT']/fluxo['Receita Operacional']
        tabela_projecao['Depreciacao e amortizacao'] = fluxo['Depreciacao e amortizacao']/fluxo['Receita Operacional']
        tabela_projecao['CAPEX'] = fluxo['CAPEX']/fluxo['Receita Operacional']
        tabela_projecao['Capital de Giro'] = fluxo['Capital de Giro Liquido']/fluxo['Receita Operacional']

        tabela_projecao = tabela_projecao[['Ano','Receita', 'EBIT', 'Depreciacao e amortizacao', 'CAPEX', 'Capital de Giro']]

        ###CALCULAR MÉDIAS###
        media_ebit = round(tabela_projecao.query(f'Ano > {self.anoInicial}')['EBIT'].mean(),2)
        media_depre_amort = round(tabela_projecao.query(f'Ano > {self.anoInicial}')['Depreciacao e amortizacao'].mean(),2)
        media_capital_giro =round(tabela_projecao.query(f'Ano > {self.anoInicial}')['Capital de Giro'].mean(),2)
        media_capex = round(tabela_projecao.query(f'Ano > {self.anoInicial}')['CAPEX'].mean(),2)

        print(
            f'Média EBIT: {round(media_ebit*100,2)}% da Receita\n'
            f'Média Depreciacao e amortizacao: {round(media_depre_amort*100,2)}% da Receita\n'
            f'Média Capital de Giro: {round(media_capital_giro*100,2)}% da Receita\n'
            f'Média CAPEX: {round(media_capex*100,2)}% da Receita'
            )

        #calcular média de crescimento da receita nos anos consolidados

        #calculando crescimento da receita
        for linha in range(1, len(fluxo)):      
            receita = float(fluxo.loc[linha, 'Receita Operacional'])
            crescimento_receita = float(fluxo.loc[linha, 'Receita Operacional'])/float(fluxo.loc[linha-1, 'Receita Operacional'])-1
            fluxo.loc[linha, 'Crescimento da Receita'] = round(crescimento_receita,4) if crescimento_receita else None

        #calculando média
        media_crescimento = round(fluxo['Crescimento da Receita'].mean(),4)      
        print(f'Média de crescimento da receita: {media_crescimento*100}% \n')

        
        self.media_ebit = media_ebit
        self.media_depre_amort = media_depre_amort
        self.media_capital_giro = media_capital_giro
        self.media_capex = media_capex
        self.media_crescimento = media_crescimento
        self.fluxo = fluxo

    def calculoWACC(self):
        
        ###IMPORTAR CSV COM PARÂMETROS PARA O WACC###
        parametros = pd.read_csv('parametros_beta (1).csv', sep=';', encoding='utf-8')
        
        beta_desalavancado = float(parametros.loc[0,'beta_desalavancado'].replace(',','.'))
        divida_liquida = float(parametros.loc[0,'divida_liquida'].replace(',','.'))
        valor_mercado = float(parametros.loc[0,'valor_mercado'].replace(',','.'))
        risk_free = float(parametros.loc[0,'risk_free'].replace(',','.'))
        equity_risk_premium = float(parametros.loc[0,'equity_risk_premium'].replace(',','.'))
        country_risk_premium = float(parametros.loc[0,'country_risk_premium'].replace(',','.'))
        rating = float(parametros.loc[0,'rating'].replace(',','.'))
        cpi = float(parametros.loc[0,'cpi'].replace(',','.'))
        ipca = float(parametros.loc[0,'ipca'].replace(',','.'))
        
        ###CALCULAR BETA, CUSTO DE CAPITAL E WACC###

        '''
           BETA ALAVANCADO (beta_a) = beta desalavancado [1 + (B/S) * (1-0.34)] 
           ;
           beta desalavancado: setor Retail, no site do Prof.Damodaran
           B = dívida líquida
           S = valor de mercado
        '''
        beta_a = round(beta_desalavancado * ((1+divida_liquida/valor_mercado)*(1-self.tributacao)),4)      

        '''
           CUSTO DE CAPITAL PRÓPRIO (rs) = risk_free + beta_a * equity risk premium + country risk premium 
           ;
           risk free: t-bonds10 em 29/07/2022 obtido no site do FED https://fred.stlouisfed.org/series/DGS10
           equity risk premium: (= rm-rf), obtido no site do Prof.Damodaran (CONFIRMAR)
           country risk premium: obtido no site do Prof.Damodaran (CONFIRMAR)           
          
           CUSTO DE CAPITAL DE TERCEIROS (rb) = risk free + country risk premium + rating da companhia
           ;
           rating: spread para o rating AAA, emitido pela S&P em Abril/2022.

           CONVERTENDO PARA REAL:
           rsR: (1+rsR)/(1+inflacaoR$)=(1+rs)/(1+inflacaoUSD)
           rbR: (1+rbR)/(1+inflacaoR$)=(1+rb)/(1+inflacaoUSD)
           ;
           cpi: consumer prince index acumulado 1 ano para jun/22 em https://www.bls.gov/cpi/ (CONFIRMAR)
           ipca: IPCA acumulado 1 ano para jun/22 (CONFIRMAR)           


        '''
        rs = round(risk_free + beta_a * equity_risk_premium + country_risk_premium,4)           
        rb = round(risk_free + country_risk_premium + rating,4)                 
        rsR = round(((1+rs)*(1+ipca)/(1+cpi))-1,4)      
        rbR = round(((1+rb)*(1+ipca)/(1+cpi))-1,4)      

        '''
           WACC = B/(S+B) * Rb * (1-t) + S/(S+B) * Rs
           ;
           B = dívida líquida
           S = valor de mercado
           Rb = custo de capital de terceiros
           Rs = custo de capital próprio
           t = tributação = 0.34

        '''
        wacc = divida_liquida/(valor_mercado+divida_liquida) * rbR * (1-self.tributacao) + valor_mercado/(valor_mercado+divida_liquida) * rsR    
        wacc = round(wacc/100,4)

        print(
            f'Beta alavancado: {round(beta_a*100,4)}%\n'
            f'Custo de capital próprio (R$): {rsR}%\n'
            f'Custo de capital de terceiros (R$): {rbR}%\n'
            f'WACC: {wacc*100}%\n'
            )


        self.divida_liquida = divida_liquida
        self.wacc = wacc

    def calculoPrecoJusto(self):

        ###CALCULAR PROJEÇÕES DFP###

        #definindo ano máximo da projeção
        lista_anos = []
        for ano in range(1,self.nAnosProjetados+1):
            lista_anos.append(self.anoFinal+ano)
        proximos_anos = pd.DataFrame(lista_anos, columns=['Ano'])

        #adicionando próximos anos e arrumando o dataframe
        projecao = pd.concat([proximos_anos, self.fluxo]).reset_index(drop = True)       
        projecao['Ano'] = projecao['Ano'].apply(lambda x: int(x))           
        projecao.sort_values(by='Ano', ascending=True, inplace=True)
        projecao.reset_index(drop=True, inplace=True)

        #calculando receita projetada
        if self.crescimentoReceita:
            media_crescimento = self.crescimentoReceita
        else:
            media_crescimento = self.media_crescimento
        for linha in range(8, len(projecao)):
            receita = float(projecao.loc[linha-1, 'Receita Operacional'])*(1+media_crescimento)
            ebit = receita*self.media_ebit
            depr_amort = receita*self.media_depre_amort
            capex = receita*self.media_capex  
            capital_giro = receita*self.media_capital_giro

            projecao.loc[linha, 'Receita Operacional'] = round(receita,3)
            projecao.loc[linha, 'EBIT'] = round(ebit,3)
            projecao.loc[linha, 'Depreciacao e amortizacao'] = round(depr_amort,3)
            projecao.loc[linha, 'CAPEX'] = round(capex,3)
            projecao.loc[linha, 'Capital de Giro Liquido'] = round(capital_giro,3)

        projecao = projecao[['Ano', 'Receita Operacional', 'EBIT', 'Depreciacao e amortizacao',
                            'CAPEX', 'Capital de Giro Liquido', 'Variacao do Capital de Giro']]

        #calculando variação do capital de giro
        for linha in range(8, len(projecao)):
            projecao.loc[linha, 'Variacao do Capital de Giro'] = projecao.loc[linha, 'Capital de Giro Liquido'] - projecao.loc[linha-1, 'Capital de Giro Liquido'] 
        
        #calcular FCFF
        projecao['FCFF'] = (projecao['EBIT']*(1-self.tributacao))+projecao['Depreciacao e amortizacao']-projecao['CAPEX']-projecao['Variacao do Capital de Giro']

        ###PROJEÇÕES DE INFLAÇÃO E PIB e CRESCIMENTO NA PERPETUIDADE###
        inflacao = self.inflacao
        crescimento_pib = self.crescimento_pib
        crescimento_perpetuidade = inflacao + crescimento_pib

        print(
            f'Inflação projetada: {inflacao*100}%\n'
            f'Crescimento PIB projetado: {crescimento_pib*100}%\n'
            f'Crescimento na perpetuidade: {crescimento_perpetuidade*100}%\n'
            )

        ###CALCULAR VALOR DA FIRMA###
        vp_acumulado = 0
        for linha in range(8, len(projecao)-1):
            mult = 1
            vp_fcff = projecao.loc[linha, 'FCFF']/(1+self.wacc)**mult
            mult += 1
            vp_acumulado += vp_fcff

        vp_perpetuo = projecao.loc[len(projecao)-1, 'FCFF']/(( self.wacc - crescimento_perpetuidade)*(1 + self.wacc)**5)

        vp_firma = vp_acumulado + vp_perpetuo

        print(
            f'VP Acumulado (milhões): R$ {round(vp_acumulado,2)}\n'
            f'VP Perpétuo (milhões): R$ {round(vp_perpetuo,2)}\n'
            f'VP Firma (milhões): R$ {round(vp_firma,2)}\n'
        )

        valor_justo = vp_firma - self.divida_liquida

        valor_justo

        numero_de_acoes = 6748926848/1000000

        preco_por_acao = valor_justo / numero_de_acoes

        print(f'\t\t\tPREÇO JUSTO: R$ {preco_por_acao.round(2)}')


In [39]:
if __name__ == '__main__':
  print('\t\t\tCENÁRIO 1')
  Valuation(inflacao_projetada=0.026, crescimento_pib=0.013, ano_inicial = 2016)
  Valuation(inflacao_projetada=0.026, crescimento_pib=0.013, crescimento_receita=0.05)
  print('\n\t\t\tCENÁRIO 2')
  Valuation(inflacao_projetada=0.026, crescimento_pib=0.013, crescimento_receita=0.1)
  print('\n\t\t\tCENÁRIO 3')
  Valuation(inflacao_projetada=0.026, crescimento_pib=0.013, crescimento_receita=0.025)

			CENÁRIO 1
	INFORMAÇÕES DFP

Ano inicial: 2016
Ano final: 2021

Média EBIT: 27.0% da Receita
Média Depreciacao e amortizacao: -2.0% da Receita
Média Capital de Giro: 17.0% da Receita
Média CAPEX: -4.0% da Receita
Média de crescimento da receita: 19.16% 

	CÁLCULO WACC

Beta alavancado: 61.9%
Custo de capital próprio (R$): 14.0856%
Custo de capital de terceiros (R$): 8.7029%
WACC: 12.41%

	CÁLCULO PREÇO JUSTO

Inflação projetada: 2.6%
Crescimento PIB projetado: 1.3%
Crescimento na perpetuidade: 3.9%

VP Acumulado (milhões): R$ 32977.17
VP Perpétuo (milhões): R$ 83658.49
VP Firma (milhões): R$ 116635.66

			PREÇO JUSTO: R$ 16.55
	INFORMAÇÕES DFP

Ano inicial: 2014
Ano final: 2021

Média EBIT: 28.0% da Receita
Média Depreciacao e amortizacao: -2.0% da Receita
Média Capital de Giro: 13.0% da Receita
Média CAPEX: -3.0% da Receita
Média de crescimento da receita: 19.16% 

	CÁLCULO WACC

Beta alavancado: 61.9%
Custo de capital próprio (R$): 14.0856%
Custo de capital de terceiros (R$): 8.702